<a href="https://colab.research.google.com/github/MaggieAppleton/Colab-Notebooks/blob/main/Anthropic_Cookbook_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anthropic Cookbook Exploration

Learning Anthropic's cookbook tactics

[Basic workflows](https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents/basic_workflows.ipynb)

## Setup

Install openai and anthropic with pip

In [ ]:
%pip install openai anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 12.8 MB/s eta 0:00:00


Get API keys from Colab secrets.  
Declare clients for Claude and OpenAI and pass in API keys.

In [ ]:
import openai
import anthropic
from google.colab import userdata
from concurrent.futures import ThreadPoolExecutor
import json
import re
from IPython.display import display, Markdown

openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')

openai_client = openai.OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)

claude_model="claude-3-5-haiku-latest"
openai_model="gpt-4.1-mini-2025-04-14"

print("✅ API clients initialized!")

✅ API clients initialized!


Setup basic functions to call Openai and Claude.  
Pass in prompts and default models.

In [ ]:
def call_openai(prompt, model=openai_model):
  try:
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=250
    )
    return response.choices[0].message.content.strip()
  except Exception as e:
    print(f"Error calling OpenAI: {e}")
    return None

def call_claude(prompt, model=claude_model):
    """
    Call Anthropic's Claude API with the given prompt
    """
    try:
        response = claude_client.messages.create(
            model=model,
            max_tokens=500,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"Error calling Claude API: {str(e)}"

print("✅ API functions ready!")

✅ API functions ready!


In [ ]:
print(f"Claude: {call_claude('What is the capital of France?')}")
print(f"Openai: {call_openai('What is the capital of France?')}")

Claude: The capital of France is Paris.
Openai: The capital of France is Paris.


Calling Claude and returning the full response object as JSON.

To make JSON work in Python, you have to import it, then use `print(response.model_dump_json(indent=2))`

Remember to return the response at the end of a function.

In [ ]:
def get_claude_response():
  try:
    response = claude_client.messages.create(
      model="claude-3-5-haiku-latest",
      max_tokens=200,
      messages=[
        {"role": "user", "content": "Who are the major thought leaders on consciousness and exercise?"}
      ]
    )
    # Print the response in nicely formatted JSON
    print(response.model_dump_json(indent=2))
    return response
  except Exception as e:
    print(f"Error calling Claude API: {str(e)}")
    return None

# Call the function and assign the returned response to a variable
response = get_claude_response()

{
  "id": "msg_01CaHAVQJqRBSbccwPx1UNbv",
  "content": [
    {
      "citations": null,
      "text": "Some notable thought leaders in consciousness and exercise include:\n\n1. Antonio Damasio\n- Neuroscientist\n- Research on consciousness, emotion, and brain function\n- Authored books like \"The Feeling of What Happens\"\n\n2. Daniel Siegel\n- Psychiatrist and mindfulness expert\n- Explores mind-body connection\n- Developed \"Interpersonal Neurobiology\" framework\n\n3. Andrew Huberman\n- Neuroscientist at Stanford\n- Studies neuroplasticity and exercise's impact on brain function\n- Popular science communicator\n\n4. Kelly McGonigal\n- Health psychologist\n- Researches mind-body interactions\n- Authored \"The Joy of Movement\"\n\n5. Martin Seligman\n- Positive psychology pioneer\n- Studied psychological benefits of physical activity\n- Explored connections between exercise and mental well-being\n\n6. John Ratey",
      "type": "text"
    }
  ],
  "model": "claude-3-5-haiku-20241022",

Display the response text in nice markdown.

In [ ]:
display(Markdown(response.content[0].text))

Some notable thought leaders in consciousness and exercise include:

1. Antonio Damasio
- Neuroscientist
- Research on consciousness, emotion, and brain function
- Authored books like "The Feeling of What Happens"

2. Daniel Siegel
- Psychiatrist and mindfulness expert
- Explores mind-body connection
- Developed "Interpersonal Neurobiology" framework

3. Andrew Huberman
- Neuroscientist at Stanford
- Studies neuroplasticity and exercise's impact on brain function
- Popular science communicator

4. Kelly McGonigal
- Health psychologist
- Researches mind-body interactions
- Authored "The Joy of Movement"

5. Martin Seligman
- Positive psychology pioneer
- Studied psychological benefits of physical activity
- Explored connections between exercise and mental well-being

6. John Ratey

In [ ]:
# General purpose function to extract the text between any given set of tags
def extract_xml(text, tag):
    """Extract content between XML tags"""
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

## Exercise 0: Pattern Recognition - Choose the Right Workflow

In [ ]:
import textwrap

def analyze_scenario(scenario_description, available_patterns=["chain", "parallel", "route"], answer_key=None, scenario_number=None):
    """
    For each scenario, decide which pattern(s) to use and explain why.
    Consider:
    - Chain: Sequential steps where each depends on the previous
    - Parallel: Multiple independent tasks that can run simultaneously
    - Route: Single input that needs different handling based on content/type

    Get user input for the choice and reasoning and return them.
    If an answer key is provided, display the corresponding answer.
    """
    print(f"Scenario: {scenario_description}")
    print("Available patterns: chain, parallel, route")

    user_choice = input("Your choice: ")
    user_reasoning = input("Reasoning: ")

    print(f"Your choice: {user_choice}")
    print(f"Reasoning: {user_reasoning}")
    print("-" * 40)

    if answer_key and scenario_number is not None:
        # Extract the relevant part of the answer key
        answer_lines = answer_key.strip().split('\n')
        start_index = -1
        end_index = -1
        for i, line in enumerate(answer_lines):
            if line.strip().startswith(f"{scenario_number}."):
                start_index = i
            elif start_index != -1 and line.strip() == "":
                end_index = i
                break
            elif start_index != -1 and i == len(answer_lines) -1:
                 end_index = i + 1

        if start_index != -1:
            print("--- Answer Key ---")
            for line in answer_lines[start_index:end_index]:
                print(line)
            print("-" * 40)


    return user_choice, user_reasoning

# Practice scenarios - analyze each one
scenarios = [
    {
        "description": "Process a research paper: extract key findings, summarize methodology, identify limitations, then create an executive summary",
        "hint": "Think about dependencies between steps"
    },

    {
        "description": "Analyze customer feedback from 50 product reviews to identify common themes and sentiment",
        "hint": "All reviews need the same type of analysis"
    },

    {
        "description": "Handle incoming customer support tickets that could be billing, technical, or account issues",
        "hint": "Different types need different specialized handling"
    },

    {
        "description": "Create a market research report: gather data on 5 competitors, analyze each one's strengths/weaknesses, then synthesize findings",
        "hint": "Consider if competitor analysis can be done independently"
    },

    {
        "description": "Process job applications: screen for basic qualifications, then route qualified candidates to department-specific evaluation",
        "hint": "This might need multiple patterns combined"
    },

    {
        "description": "Analyze a single legal document: extract key clauses, identify potential risks, assess compliance, then draft recommendations",
        "hint": "Does each step need the output of the previous step?"
    },

    {
        "description": "Handle mixed content: blog posts, social media posts, and press releases all need different style analysis",
        "hint": "Same inputs but different processing based on content type"
    },

    {
        "description": "Process survey responses: categorize by topic, then analyze sentiment within each category, then create summary report",
        "hint": "Multiple patterns might be needed in sequence"
    },

    {
        "description": "Evaluate 10 business proposals that all need: financial analysis, risk assessment, and market viability scoring",
        "hint": "Same analysis framework applied to different inputs"
    },

    {
        "description": "Handle user queries that could be: password reset, feature requests, bug reports, or general questions",
        "hint": "Different query types need different response strategies"
    }
]

answer_key = """
ANSWER KEY:

1. Research paper processing: CHAIN
   - Each step depends on the previous (can't summarize without extracting findings first)
   - Sequential dependencies make this a clear chain pattern

2. 50 product reviews analysis: PARALLEL
   - Same analysis applied to different inputs
   - Reviews can be processed independently and simultaneously

3. Customer support tickets: ROUTE
   - Different types need specialized handling
   - Content determines which expert/process should handle it

4. Market research with 5 competitors: PARALLEL then CHAIN
   - Competitor analysis can be done in parallel (independent)
   - Then chain to synthesize findings sequentially

5. Job application processing: CHAIN then ROUTE
   - First chain: screen qualifications sequentially
   - Then route qualified candidates to appropriate departments

6. Legal document analysis: CHAIN
   - Each step builds on previous analysis
   - Sequential processing ensures thorough analysis

7. Mixed content analysis: ROUTE
   - Same inputs but different processing based on content type
   - Each content type needs specialized analysis approach

8. Survey response processing: ROUTE then PARALLEL then CHAIN
   - Route: categorize by topic
   - Parallel: analyze sentiment within each category
   - Chain: create summary report from categorized results

9. 10 business proposals: PARALLEL
   - Same evaluation framework for all proposals
   - Each proposal can be evaluated independently

10. User query handling: ROUTE
    - Different query types need different response strategies
    - Content determines appropriate handling approach
"""


print("For each scenario below, determine which workflow pattern(s) would be most appropriate:")
print("CHAIN: When steps must be done sequentially, each building on the previous")
print("PARALLEL: When multiple independent tasks can be done simultaneously")
print("ROUTE: When input needs different handling based on its type/content")
print("\n")

# Store the user's responses
user_responses = {}

for i, scenario in enumerate(scenarios, 1):
    print(f"SCENARIO {i}:")
    choice, reasoning = analyze_scenario(scenario["description"], answer_key=answer_key, scenario_number=i)
    user_responses[f"Scenario {i}"] = {"choice": choice, "reasoning": reasoning}
    print(f"Hint: {scenario['hint']}")
    print("\n")

# Optionally, display all collected responses at the end
print("--- Your Recorded Responses ---")
for scenario, response in user_responses.items():
    print(f"{scenario}:")
    print(f"  Choice: {response['choice']}")
    print(f"  Reasoning: {response['reasoning']}")
    print("-" * 20)

In [ ]:
# Reset answers
user_responses = {}
print("user_responses has been cleared.")

user_responses has been cleared.


## Prompt Chaining

Break a task down into substeps. Each step builds on previous results. Valuable in scenarios requiring multi-step reasoning, iterative refinement, or complex information processing.

In [ ]:
# Chaining pattern

from typing import List

def chain(input: str, prompts: List[str]) -> str:
  """Chain LLM calls. Pass results between steps."""
  result = input
  for i, prompt in enumerate(prompts, 1):
    print(f"\n-----Step {i}-----")
    print(f"\nInput: {result}")
    print(f"\nPrompt: {prompt}")
    result = call_claude(f"\n{prompt}\nInput: {result}")
    print(f"\nResult:\n{result}")
  return result

Useful for:
- Document Q&A. First extract relevant quotes, then use those as context to generate an answer
- Review and refine generated answers. Fact-check to reduce hallucinations.
- Debug generated code.
- Multi-step problem solving: identify the problem, make a plan, solve each step sequentially.
- Data processing: extract information from text, then format the information in a specific style.

In [ ]:
# Example usage of the chain function

input = "Margaret Thatcher slashed government benefits, leaving Britain's most vulnerable citizens impoverished and angry."

prompts = [
    "Extract the key nouns and verbs from the following sentence. Return them as a comma-separated list.",
    "Using the keywords provided, write a short, snippy tweet. Use less than 280 characters.",
    "Write an ungenerous reply to this tweet. Use less than 280 characters."
]

# Call the chain function
final_output = chain(input, prompts)


-----Step 1-----

Input: Margaret Thatcher slashed government benefits, leaving Britain's most vulnerable citizens impoverished and angry.

Prompt: Extract the key nouns and verbs from the following sentence. Return them as a comma-separated list.

Result:
Nouns: Margaret Thatcher, government, benefits, Britain, citizens
Verbs: slashed, leaving, impoverished

-----Step 2-----

Input: Nouns: Margaret Thatcher, government, benefits, Britain, citizens
Verbs: slashed, leaving, impoverished

Prompt: Using the keywords provided, write a short, snippy tweet. Use less than 280 characters.

Result:
Thatcher slashed government benefits, leaving Britain's citizens impoverished. Her legacy? A country where the vulnerable suffer while the wealthy celebrate. #ThatcherismFailed

-----Step 3-----

Input: Thatcher slashed government benefits, leaving Britain's citizens impoverished. Her legacy? A country where the vulnerable suffer while the wealthy celebrate. #ThatcherismFailed

Prompt: Write an unge

## Exercise 1: Basic Chain Implementation

In [ ]:
def chain(input_text, prompts):
    result = input_text
    for i, prompt in enumerate(prompts, 1):
      print(f"--- Step {i} ---")
      print(f"Input: {result}")
      result = call_claude(f"\n{prompt}\nInput:{result}")
      print(f"Result: {result}\n")
      print("-" * 40)
    return result

# Test data for Exercise 1
email_text = """
From: sarah.johnson@techcorp.com
To: support@ourcompany.com
Subject: Urgent: Server downtime affecting production

Hi team,

We're experiencing severe server issues that started around 2 PM EST.
Our production environment is completely down, affecting approximately
500 users. The error logs show database connection timeouts.

We need immediate assistance. This is blocking our quarterly release.

Please escalate to your senior team.

Best regards,
Sarah Johnson
Technical Lead, TechCorp
"""

email_processing_steps = [
    "Extract the key information: sender, urgency level, problem description, and impact.",
    "Classify the urgency (low/medium/high/critical) and explain your reasoning.",
    "Suggest 3 immediate action items to resolve this issue.",
    "Draft a professional response acknowledging the issue and next steps."
]

# Uncomment to test your implementation
result = chain(email_text, email_processing_steps)
print(f"\nFinal result:\n{result}")

--- Step 1 ---
Input: 
From: sarah.johnson@techcorp.com
To: support@ourcompany.com
Subject: Urgent: Server downtime affecting production

Hi team,

We're experiencing severe server issues that started around 2 PM EST. 
Our production environment is completely down, affecting approximately 
500 users. The error logs show database connection timeouts.

We need immediate assistance. This is blocking our quarterly release.

Please escalate to your senior team.

Best regards,
Sarah Johnson
Technical Lead, TechCorp

Result: Here's the extracted key information:

Sender: Sarah Johnson (sarah.johnson@techcorp.com)

Urgency Level: High/Critical (indicated by "Urgent" in subject line and phrases like "severe", "need immediate assistance", "please escalate")

Problem Description: Server issues causing database connection timeouts, leading to complete production environment downtime

Impact: 
- Production environment is down
- Approximately 500 users affected
- Blocking quarterly product release



## Exercise 2: Chain with Error Handling

In [ ]:
def robust_chain(input_text, prompts, max_retries=2):
    """
    Implement chain with error handling
    - If a step fails (including validation failure), retry up to max_retries times
    - If still fails, use previous result and continue
    - Track which steps had errors
    """
    result = input_text
    error_steps = []

    for i, prompt in enumerate(prompts, 1):
      print(("-" * 15) + f" Step {i} " + ("-" * 15))
      print(f"Input: {result}\n")

      success = False
      retries = 0
      step_result = result # Store the result before the current step

      while not success and retries <= max_retries:
        try:
          response = call_claude(f"\n{prompt}\nInput: {step_result}") # Use step_result as input
          if response is None or response.strip() == "":
            raise ValueError("Empty or null string response from LLM")

          qa_check_response = validator(prompt, step_result, response) # Pass prompt, input, and output
          print(f"QA Check Response: {qa_check_response}")

          if "FAIL" in qa_check_response:
            raise ValueError(f"QA check failed: {qa_check_response}") # Treat QA failure as an error

          print(f"Passed QA check for Step {i}")
          success = True
          step_result = response # Update step_result only on success

        except Exception as e:
          print(f"Error at step {i} (Retry {retries + 1}/{max_retries + 1}): {e}")
          retries += 1

      if not success:
        print(f"Step {i} failed after {max_retries} retries. Continuing with previous result.")
        error_steps.append(i)
        # The result remains the same as the input for this failed step
      else:
        result = step_result # Update the main result only if the step was successful


      print(f"Result: {result}\n") # Print the result used for the next step or the final result

    print("--- Final Run Summary ---")
    if error_steps:
        print(f"Steps with errors: {error_steps}")
    else:
        print("All steps completed successfully.")
    print("-" * 25)

    return result

def validator(prompt, input, output):
  """Check whether the prompts and outputs are valid using an LLM call."""

  qa_prompt = f"""
  You are a helpful QA agent who evaluates both user input and LLM outputs.
  Given this user input and LLM output, check whether we need to flag an issue. It's an issue if either the prompt or output does not make sense given the input for that specific step.

  Prompt: {prompt}
  Input: {input}
  Output: {output}

  Reply with PASS if everything makes sense and is appropriate for this step. Reply with FAIL:prompt or FAIL:output depending on which one you think has an issue. Explain why you failed the prompt or output and return it in a set of <reasoning> tags.
  """
  response = call_claude(qa_prompt)
  return response


# Test with potentially problematic prompts
tricky_prompts = [
    "Summarize this text in exactly 50 words.",
    "Extract all numbers and convert to percentages.",
    "This is a deliberately confusing prompt that might cause issues: @@#$%",
    "Create a final summary of the processed information."
]

test_text = "Sales increased 25% this quarter. We had 1200 new customers and revenue of $450,000."

print("Testing error handling with tricky prompts and validation...\n")
# Uncomment to test
result = robust_chain(test_text, tricky_prompts)

Testing error handling with tricky prompts and validation...

--------------- Step 1 ---------------
Input: Sales increased 25% this quarter. We had 1200 new customers and revenue of $450,000.

QA Check Response: FAIL:output

<reasoning>
The output does not meet the specific requirement of exactly 50 words. The provided summary is 62 words long, which violates the original prompt's instruction to summarize the text in exactly 50 words. While the content is relevant and accurately reflects the input, the length is incorrect and therefore fails to meet the task's constraints.
</reasoning>
Error at step 1 (Retry 1/3): QA check failed: FAIL:output

<reasoning>
The output does not meet the specific requirement of exactly 50 words. The provided summary is 62 words long, which violates the original prompt's instruction to summarize the text in exactly 50 words. While the content is relevant and accurately reflects the input, the length is incorrect and therefore fails to meet the task's const

## Test: Write your own 3 step chain scenario

## Parallelisation

Multiple LLMs run calls at the same time

In [ ]:
# Parallel pattern

def parallel(prompt, inputs, n_workers=3):
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\\n\\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]

## ✅ Exercise 3: Basic Parallelisation

In [ ]:
def parallel(prompt, inputs, n_workers=3):
  with ThreadPoolExecutor(max_workers=n_workers) as executor:
    futures = [executor.submit(call_claude, f"{prompt}\\n\\nInput: {x}") for x in inputs]
    return [f.result() for f in futures]


# Test data for Exercise 3
product_reviews = [
    "This product is amazing! Great quality and fast shipping. Highly recommend.",
    "Terrible quality. Broke after one week. Customer service was unhelpful.",
    "Decent product for the price. Nothing special but does the job.",
    "Outstanding! Exceeded expectations. Will buy again.",
    "Mixed feelings. Good features but poor documentation."
]

sentiment_prompt = """
Analyze the sentiment of this review and extract key themes.
Format your response as:
Sentiment: [Positive/Negative/Neutral]
Key themes: [list 2-3 themes]
Score: [1-10]
"""

print("Product reviews to analyze:")
for i, review in enumerate(product_reviews, 1):
    print(f"{i}. {review}")

print(f"\nPrompt to apply to each: {sentiment_prompt}")

reviews = parallel(sentiment_prompt, product_reviews)
for i, review in enumerate(reviews, 1):
    print(f"\nReview {i} Analysis:")
    print(review)


summary_prompt = f"""
Write a short 1-2 sentence summary of how people have reviewed this product. Mention any key positives or negatives that are notable or repeated across the reviews.
Reviews: {reviews}
"""

def summarise(content):
  response = call_claude(content)
  return response

display(f"Review summary: {summarise(summary_prompt)}")

Product reviews to analyze:
1. This product is amazing! Great quality and fast shipping. Highly recommend.
2. Terrible quality. Broke after one week. Customer service was unhelpful.
3. Decent product for the price. Nothing special but does the job.
4. Outstanding! Exceeded expectations. Will buy again.
5. Mixed feelings. Good features but poor documentation.

Prompt to apply to each: 
Analyze the sentiment of this review and extract key themes.
Format your response as:
Sentiment: [Positive/Negative/Neutral]
Key themes: [list 2-3 themes]
Score: [1-10]


Review 1 Analysis:
Sentiment: Positive
Key themes: 
- Product quality
- Fast shipping
- Strong recommendation
Score: 9/10

The review is overwhelmingly positive, using enthusiastic language like "amazing" and "highly recommend". The customer highlights two specific positive attributes (quality and shipping speed) and seems very satisfied with their purchase.

Review 2 Analysis:
Sentiment: Negative
Key themes:
- Product durability/quality

'\nReview summary: Based on these reviews, the product appears to receive mixed feedback, with some customers extremely satisfied and praising its quality and shipping, while others experienced durability issues and poor customer support. Overall, the reviews suggest a product that ranges from highly recommended to disappointingly average, with scores varying from 2/10 to 9/10.'

## ✅ Exercise 4: Parallel with Different Prompts

In [ ]:
# TODO: Implement parallel processing with different prompts
    # - Take a list of (prompt, input) tuples
    # - Process each pair concurrently
    # - Return results in same order

def parallel_different_prompts(prompt_input_pairs, n_workers=3):

  # Get prompts
  prompts = []
  for pair in prompt_input_pairs:
    prompts.append(pair[0])
  # print(f"Prompts: {prompts}")

  # Get input
  input = analysis_tasks[0][1]
  # print(f"Input: {input}")

  with ThreadPoolExecutor(max_workers=n_workers) as executor:
    futures = [executor.submit(call_claude, f"{prompt}\\n\\nInput: {input}") for prompt in prompts]
    return [f.result() for f in futures]


# Test data - analyzing a business scenario from different perspectives
business_scenario = """
TechStart Inc. is considering launching a new AI-powered mobile app.
Initial development costs are $500K, monthly operating costs $50K.
Market research shows 100K potential users in year 1, growing to 500K by year 3.
Subscription price would be $9.99/month. Main competitors charge $12-15/month.
Team has strong technical skills but limited marketing experience.
"""

analysis_tasks = [
    ("Analyze this from a financial perspective. Calculate ROI and break-even point.", business_scenario),
    ("Evaluate the competitive landscape and market positioning.", business_scenario),
    ("Assess the technical and operational risks.", business_scenario),
    ("Recommend a marketing strategy for launch.", business_scenario)
]

print("Business scenario:")
print(business_scenario)
print("\nAnalysis tasks:")
for i, (prompt, _) in enumerate(analysis_tasks, 1):
    print(f"{i}. {prompt}")

# Uncomment to test
results = parallel_different_prompts(analysis_tasks)
for i, result in enumerate(results, 1):
    print(f"\nAnalysis {i}:")
    print(result)

Business scenario:

TechStart Inc. is considering launching a new AI-powered mobile app. 
Initial development costs are $500K, monthly operating costs $50K.
Market research shows 100K potential users in year 1, growing to 500K by year 3.
Subscription price would be $9.99/month. Main competitors charge $12-15/month.
Team has strong technical skills but limited marketing experience.


Analysis tasks:
1. Analyze this from a financial perspective. Calculate ROI and break-even point.
2. Evaluate the competitive landscape and market positioning.
3. Assess the technical and operational risks.
4. Recommend a marketing strategy for launch.
Prompts: ['Analyze this from a financial perspective. Calculate ROI and break-even point.', 'Evaluate the competitive landscape and market positioning.', 'Assess the technical and operational risks.', 'Recommend a marketing strategy for launch.']
Input: 
TechStart Inc. is considering launching a new AI-powered mobile app. 
Initial development costs are $500K,

## Routing

In [ ]:
# Routing pattern

support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly

    Keep responses professional but friendly.

    Input: """,

    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant

    Use clear, numbered steps and technical details.

    Input: """,

    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes

    Maintain a serious, security-focused tone.

    Input: """,

    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage

    Be educational and encouraging in tone.

    Input: """
}

def route(input_text, routes):
    selector_prompt = f'''
    Analyze the input and select the most appropriate handler from these options: {list(routes.keys())}

    <reasoning>
    Explain why this input should be routed to a specific handler.
    </reasoning>

    <selection>
    The chosen handler name
    </selection>

    Input: {input_text}
    '''

    # list(routes.keys()) will print ['billing', 'technical', 'account', 'product']

    route_response = call_claude(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()

    print(f"Routing reasoning: {reasoning}")
    print(f"Selected route: {route_key}")

    if route_key in routes:
        selected_prompt = routes[route_key]
        return call_claude(f"{selected_prompt}\\n\\nInput: {input_text}")
    else:
        return "Error: Could not determine appropriate route"

response = route("I need a refund", support_routes)
print(response)

Routing reasoning: The input "I need a refund" is most likely related to billing issues. A refund request typically involves financial transactions and compensation, which falls directly under the billing department's responsibilities. They would handle processing refunds, investigating the reason for the refund, and managing the financial aspects of returning money to a customer.
Selected route: billing
Billing Support Response:

Thank you for reaching out about a potential refund. I understand you're seeking a refund, but I'll need a bit more information to assist you effectively. Could you please provide:

1. The specific reason for your refund request
2. The date of the charge
3. The amount in question
4. The product or service related to the charge

Our team is committed to resolving billing matters promptly and fairly. Once I have more details, I'll be happy to guide you through our refund process and help you resolve this matter.


## ✅ Exercise 5: Basic Routing Pattern

In [ ]:
def route(input_text, routes):
    selector_prompt = f'''
    Analyse the input and select the most appropriate handler from these options: {list(routes.keys())}

    Explain why this input should be given to this specific handler and wrap it in <reasoning> tags.

    Give your selected handler inside <selection> tags.

    Input: {input_text}
    '''

    route_response = call_claude(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()

    print(f"Reasoning: {reasoning}")
    print(f"Route: {route_key}")

    if route_key in routes:
      selected_prompt = routes[route_key]
      return call_claude(f"{selected_prompt}\nInput: {input_text}")
    else:
      return "Error: Could not pick an appropriate route"

# Test data for Exercise 5
customer_inquiries = [
    "I can't log into my account. I keep getting an error message.",
    "Why was I charged $99 when I thought my plan was $49?",
    "How do I export my data to CSV format?",
    "I think someone accessed my account without permission."
]

support_routes = {
    "technical": """You are a technical support specialist. Provide step-by-step troubleshooting instructions.
    Start with basic checks, then advanced solutions. Include system requirements if relevant.""",

    "billing": """You are a billing support agent. Explain charges clearly and provide resolution steps.
    Always reference specific billing periods and amounts. Offer payment options when appropriate.""",

    "account": """You are an account security specialist. Prioritize security verification and protection.
    Provide clear steps for account recovery. Include security recommendations.""",

    "product": """You are a product specialist. Focus on feature education and best practices.
    Provide examples and link to documentation. Suggest related features that might help."""
}

print("Customer inquiries to route:")
for i, inquiry in enumerate(customer_inquiries, 1):
    print(f"{i}. {inquiry}")

print(f"\nAvailable routes: {list(support_routes.keys())}")

# Uncomment to test your implementation
for i, inquiry in enumerate(customer_inquiries, 1):
    print(f"\nInquiry {i}: {inquiry}")
    response = route(inquiry, support_routes)
    print(f"Response: {response}")

Customer inquiries to route:
1. I can't log into my account. I keep getting an error message.
2. Why was I charged $99 when I thought my plan was $49?
3. How do I export my data to CSV format?
4. I think someone accessed my account without permission.

Available routes: ['technical', 'billing', 'account', 'product']

Inquiry 1: I can't log into my account. I keep getting an error message.
Reasoning: This input describes a login issue with an account, which suggests a technical problem related to accessing the user's account. The user is experiencing an error message when attempting to log in, which points directly to a technical support need. A technical support representative would be best equipped to help diagnose login problems, check for system errors, reset credentials, or troubleshoot authentication issues.
Route: technical
Response: Technical Support Troubleshooting Guide: Account Login Issues

System Requirements Check:
- Supported browsers: Chrome, Firefox, Safari, Edge (lates

## ✅ Exercise 6: Advanced Routing with Confidence

In [ ]:
# TODO: Implement routing with confidence scoring
#     - Get the LLM to provide a confidence score (0-1) for its route selection
#     - If confidence is below threshold, route to a "general" handler
#     - Extract both reasoning and confidence from the response

def route_with_confidence(input_text, routes, confidence_threshold=0.7):
    selector_prompt = f'''
    Analyze the following query and select the appropriate handler to route the request to: {list(routes.keys())}

    <selection>
    Your selected handler
    </selection>

    <reasoning>
    Explain your reasoning for routing to this handler
    </reasoning>

    <confidence>
    Your confidence level that this is the right handler for this query as a score from 0-1
    </confidence>

    Query: {input_text}
    '''

    selection_response = call_claude(selector_prompt)
    route_selection = extract_xml(selection_response, 'selection')
    route_reasoning = extract_xml(selection_response, 'reasoning')
    route_confidence_str = extract_xml(selection_response, 'confidence') # Extract as string
    route_confidence = float(route_confidence_str) # Convert string to number

    print(f"Selection: {route_selection}")
    print(f"reasoning: {route_reasoning}")
    print(f"confidence: {route_confidence}")

    if route_selection in routes.keys() and route_confidence > confidence_threshold:
      selected_prompt = routes[route_selection]
      response = call_claude(f"{selected_prompt}\nInput:{input_text}")
      return response
    else:
      # If the confidence level is too low OR selected route is not in the defined routes, default to general
      print(f"Confidence level too low, routing to general")
      selected_prompt = routes['general']
      response = call_claude(f"{selected_prompt}\nInput:{input_text}")
      return response


# Add a general route for low-confidence cases
enhanced_routes = support_routes.copy()
enhanced_routes["general"] = """You are a general support agent. Provide helpful guidance and
determine if the customer needs to be transferred to a specialist. Ask clarifying questions if needed."""

ambiguous_queries = [
    "I need help with my account.",
    "I don't like this.",
    "Can I talk to Gary",
    "The app isn't working right and I'm being charged."
]

print("Ambiguous inquiries to test confidence routing:")
for i, inquiry in enumerate(ambiguous_queries, 1):
    print(f"{i}. {inquiry}")

for i, query in enumerate(ambiguous_queries, 1):
  print(f"\nQuery {i}: {query}")
  response = route_with_confidence(query, enhanced_routes)
  print(f"Response: {response}")

Ambiguous inquiries to test confidence routing:
1. I need help with my account.
2. I don't like this.
3. Can I talk to Gary
4. The app isn't working right and I'm being charged.

Query 1: I need help with my account.
Selection: account
reasoning: The query directly mentions "my account", which strongly indicates that the account handler is the most appropriate choice. This handler would be best equipped to assist with account-related issues, such as account management, account details, or account-specific concerns.
confidence: 0.9
Response: I'll help you with your account securely. To proceed, I'll need to verify your identity and guide you safely through the account recovery process. 

First, let's establish some secure verification steps:

1. Identity Verification
- Do you have access to the email associated with the account?
- Can you provide the last 4 digits of the registered phone number?
- Do you remember your security questions?

2. Security Recommendations
- Use a strong, uniq

## ✅ Exercise 7: Combined Workflows

In [ ]:
# Implement a complex workflow that combines all three patterns:
  # 1. Route each response to appropriate analysis type (satisfaction/complaint/suggestion)
  # 2. Process responses of the same type in parallel
  # 3. Chain the results through summary and recommendation steps

survey_data = [
    "The new interface is confusing. I can't find basic features anymore.",
    "Love the recent updates! The app is much faster now.",
    "Pricing is too high compared to competitors. Consider a discount tier.",
    "Customer service was excellent. Quick response and helpful.",
    "The mobile app crashes frequently on my device.",
    "Great product overall but needs better documentation.",
    "Would like to see integration with more third-party tools.",
    "Billing process is complicated and unclear."
]

survey_routes = {
      "satisfaction": "Briefly analyze this positive feedback for key satisfaction drivers.",
      "complaint": "Briefly analyze this complaint for root causes and impact severity.",
      "suggestion": "Briefly analyze this suggestion for feasibility and business value."
    }

from collections import defaultdict

def route_response(input):
    selector_prompt = f'''
    Analyse this customer feedback and select an appropriate handler to route it to: {list(survey_routes.keys())}

    <selection>
    Your selected handler
    </selection>

    <reasoning>
    Very briefly explain why you selected this handler for this feedback
    </reasoning>

    Feedback: {input}
    '''

    route_response = call_claude(selector_prompt)
    route_selection = extract_xml(route_response, 'selection').strip().lower()
    route_reasoning = extract_xml(route_response, 'reasoning')

    # print(f"--------- Routing process ----------")
    # print(f"Reasoning: {route_reasoning}")
    # print(f"Selection: {route_selection}")

    return route_selection

def process_parallel_prompts(prompt, inputs, n_workers=3):
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(call_claude, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]

def summarise_responses(responses):
  # For each item in responses, get the category (key), and get each piece of user feedback, and its corresponding response

  feedback_and_responses = ''

  for category_responses in responses.values():
    for response_pair in category_responses:
        feedback, analysis = response_pair
        feedback_and_responses += f"<feedback>{feedback}</feedback>\n"
        feedback_and_responses += f"<analysis>{analysis}</analysis>\n\n"

  prompt = f"""
  Summarise this set of customer feedback and their corresponding analyses into a few paragraphs. Be specific about what customers like, dislike, and want to change. Quote from the responses where possible.

  {feedback_and_responses}
  """

  summary = call_claude(prompt)
  print(f"\n –––––––––– Summary: –––––––––– \n {summary} ")
  return summary

def make_recommedations(summary):
  prompt = f"""
  You are an experienced product manager. Here is a summary of what we learned from our most recent customer survery. Write a report recommending what we should do next.
  """
  recommendations = call_claude(f"{prompt}\nSummary: {summary}")
  return recommendations

def process_survey_responses(responses):

    # Responses is a list of strings
    # print(f"Responses: {responses}")

    # Classify responses by routing to the handler
    classified_responses = []
    for response in responses:
      classification = route_response(response)
      classified_responses.append([response, classification])

    # print(f"Classified responses: {classified_responses}")

    # Process responses in parallel
    # defaultdict helps group items
    grouped_classified_responses = defaultdict(list)
    support_replies = {}

    for response, classification in classified_responses:
      grouped_classified_responses[classification].append(response)

    for classification, responses in grouped_classified_responses.items():
      prompt = survey_routes[classification]
      results = process_parallel_prompts(prompt, responses)
      # print('-' * 20 + "Responses by classification" + '-' * 20)

      support_replies[classification] = []
      for response, result in zip(responses, results):
        support_replies[classification].append([response, result])

      # print(f"Categorised support replies: {support_replies}")

    # Summarise all responses
    summary = summarise_responses(support_replies)

    # Make recommendations of what to do next based on summary
    recommendations = make_recommedations(summary)
    return recommendations


# Print the survey responses
print("Survey responses to process:")
for i, response in enumerate(survey_data, 1):
    print(f"{i}. {response}")

# Print the final report
result = process_survey_responses(survey_data)
print(f"\n –––––––––– Final Analysis Report: –––––––––– \n{result}")

Survey responses to process:
1. The new interface is confusing. I can't find basic features anymore.
2. Love the recent updates! The app is much faster now.
3. Pricing is too high compared to competitors. Consider a discount tier.
4. Customer service was excellent. Quick response and helpful.
5. The mobile app crashes frequently on my device.
6. Great product overall but needs better documentation.
7. Would like to see integration with more third-party tools.
8. Billing process is complicated and unclear.

 –––––––––– Summary: –––––––––– 
 Summary of Customer Feedback and Analysis:

The customer feedback reveals a mix of positive experiences and areas for improvement across the product's interface, performance, and services. 

User Experience Challenges:
Customers have expressed significant frustration with the new interface, with one user noting it is "confusing" and making it difficult to find basic features. The mobile app also suffers from technical issues, with frequent crashes th

## Exercise 8: Real-World Content Pipeline

In [ ]:
def content_pipeline(topic, content_types):
    """
    TODO: Build a content creation pipeline:
    1. Generate initial research and key points for the topic
    2. Create different content types in parallel (blog post, social media, email)
    3. Chain through editing and optimization steps
    4. Route final content to appropriate channels
    """
    # YOUR CODE HERE
    pass

# Test the content pipeline
test_topic = "The Future of Remote Work: Trends and Predictions for 2024"
content_types = ["blog_post", "social_media", "email_newsletter"]

print(f"Creating content pipeline for: {test_topic}")
print(f"Content types: {content_types}")

# Uncomment to test
# final_content = content_pipeline(test_topic, content_types)
# print(f"\nFinal Content Package:\n{final_content}")

## Exercise 9: Dynamic Workflow Selection

In [ ]:
def smart_processor(input_data, task_description):
    """
    TODO: Implement a system that automatically chooses the right workflow:
    - Analyze the input and task to determine if it needs chain, parallel, or routing
    - Implement the appropriate workflow dynamically
    - Handle mixed scenarios (e.g., route first, then parallel)
    """
    # YOUR CODE HERE
    pass

# Test scenarios
test_scenarios = [
    ("Analyze this single document step by step", "Single document requiring sequential processing"),
    (["Review 1", "Review 2", "Review 3"], "Multiple similar items for analysis"),
    ("Customer inquiry about billing", "Single item needing specialized handling"),
    (["Technical question", "Billing question", "Feature request"], "Mixed types requiring different handling")
]

print("Test scenarios for dynamic workflow selection:")
for i, (data, description) in enumerate(test_scenarios, 1):
    print(f"{i}. {description}")
    print(f"   Data: {data}")

# Uncomment to test
# for i, (data, description) in enumerate(test_scenarios, 1):
#     print(f"\nScenario {i}: {description}")
#     result = smart_processor(data, description)
#     print(f"Result: {result}")


## Exercise 10: Performance Monitoring

In [ ]:
def monitored_chain(input_text, prompts):
    """
    TODO: Implement chain with performance monitoring:
    - Track execution time for each step
    - Count token usage (approximate)
    - Monitor error rates
    - Log performance metrics
    """
    # YOUR CODE HERE
    pass

def monitored_parallel(prompt, inputs, n_workers=3):
    """
    TODO: Implement parallel with performance monitoring:
    - Track total execution time
    - Monitor thread utilization
    - Count successful vs failed operations
    """
    # YOUR CODE HERE
    pass

# Test performance monitoring
test_prompts = [
    "Summarize this text in 100 words.",
    "Extract key metrics and numbers.",
    "Provide recommendations based on the analysis."
]

test_text = "Q3 results show 25% revenue growth, 1000 new customers, and 95% satisfaction rate."

print("Testing performance monitoring...")
# Uncomment to test
# result = monitored_chain(test_text, test_prompts)
# print(f"Final result: {result}")